# Tarea 3:

## 1. Escriba la energía HF para el átomo de Berilio.  




## 2.  Programar la teoría del campo autoconsistente Hartree-Fock  en moleculas
## I. Repaso teórico

En el método hartree-fock-Roothaan en moleculas, se construyen los orbitales moleculares a 
partir de orbitales atómicos, centrados sobre los respectivos átomos:

$$ \psi_i = \sum_\mu C_{\mu i} \phi_{\mu}$$

Los oribtales que minimizan la energía HF son orbitales solución de las ecuación de psuedo-valores 
propios de Roothaan.

En esta tarea van escribir un código que calcula la energía variacional hartree-fock de una molecula mediante
el método del campo autoconsistente y la ecuaciones de Roothaan. A modo de repaso esta teoría busca 
resolver la ecuación matriz de pseudo valores propios:


$$\sum_{\nu} F_{\mu\nu}C_{\nu i} = \epsilon_i\sum_{\nu}S_{\mu\nu}C_{\nu i}$$
$${\bf FC} = {\bf SC\epsilon},$$

Como el operador de Fock depende explicitamente de las funciones de los diferentes orbitales, esta ecuación 
solamente se puede resolver iterativamente hasta llegar a una solución auto-consistente para la 
matriz de coeficientes orbitales **C**. La matriz de Fock **F** se construye a partir de la matriz densidad **D**
y las integrales bi-electronicas en la base de los orbitales atómicos $\{\mu\}$.

$$F_{\mu\nu} = H_{\mu\nu} + 2(\mu\,\nu\left|\,\lambda\,\sigma)D_{\lambda\sigma} - (\mu\,\lambda\,\right|\nu\,\sigma)D_{\lambda\sigma},$$

La matriz densidad $D_{\lambda\sigma}$ se construye mediante la matriz de coeficientes orbitales **C**:

$$D_{\lambda\sigma} = C_{\sigma i}C_{\lambda i}$$

La matriz de coeficientes orbitales **C** es una matriz de $N\times M$, donde $N$ es el número de funciones bases atómicas, y $M$ es el número total de orbitales moleculares. Por lo tanto la matriz **C** se puede interpretar físicamente como, la contribución de cada función base (columnas) a un determinado orbital molecular.  La matriz densidad se por lo tanto es una matriz cuadrada que describe la densidad electrónica de cada orbital molecular. 

La energía total RHF esta dada por:

$$E^{\rm RHF}_{\rm total} = E^{\rm RHF}_{\rm elec} + E^{\rm BO}_{\rm nuc},$$

Donde $E^{\rm RHF}_{\rm elec}$ es la energía electrónica RHF final y   $E^{\rm BO}_{\rm nuc}$ es la energía de repulsión núcleo-núcleo total BO.  Para calcular la energía electrónica total usamos la matiz densidad en la base AO:

$$E^{\rm RHF}_{\rm elec} = (F_{\mu\nu} + H_{\mu\nu})D_{\mu\nu},$$

y la energía de repulsión nuclear es:
$$E^{\rm BO}_{\rm nuc} = \sum_{A>B}\frac{Z_AZ_B}{r_{AB}}$$

## II. Implementación

Primero que nada debemos instalar el modulo python para Psi4 (solo Linux!). La manera más simple de 
hacerlo es instalarlo  desde una distribución anaconda. La pueden bajar en:

https://www.anaconda.com/download/#linux

Les recomiendo bajar la versión con python 2.7.
Una vez instalado, corren:

**conda create -n p4env python=2.7 psi4 psi4-rt -c psi4/label/dev -c psi4**

Esto les va a instalar los binarios de psi4 y el modulo python nececario para
esta tarea. Finalmente deben definir un directorio para el scratch. Abran su .bashrc 
y agregen la linea:

**export PSI_SCRATCH=/home/su_nombre_de_usuario/.scratch/psi4**

Ahora solo falta recargar el bashrc y activar el ambiente psi4

**source activate p4env**

ahora puede abrir jupyter-notebook e importar psi4.


In [2]:
# ==> Importar Psi4 & NumPy <==
import psi4
import numpy as np

Próximo paso es fijar las variables de memoria. Y definir la molecula a la cual le van a calcular la
energía. 

Memoria y output.  Reservar 500 MB de memoria a Psi4.
- Fijar el archivo de output como: "output.dat"
- Fijar la memoria numpy que va a usar para alamcenar los tensores.

Molecule definition:
- Definir la posición de los núcleos en una molécula de agua.
- Simetría C1

Opciones del calculo de comprarasión:
- Base y convergencia


In [23]:
# ==> Opciones Básicas Psi4 <==
# Memoria
psi4.set_memory(int(5e8))
numpy_memory = 500

# Output
psi4.core.set_output_file('output.dat', False)

# Geometría
mol = psi4.geometry("""
0 1
Be
""")

# Opciones de calculo
psi4.set_options({'basis': 'cc-pvdz',
                  'e_convergence': 1e-8})

psi4.energy('scf/cc-pvdz')

-14.57234126403459

Como van a escribir su propio código HF, vamos a especifiar primeros las variables necesarias para el ciclo 
iterativo, que son el número máximo de ciclos que van a permitir y el criterio de convergencia para el energía.

In [24]:
# Máximo de iteraciones SCF
MAXITER = 40
# Criterio de Convergencia para la energía.
E_conv = 1.0e-6
# Energía de repulsión nucelo-nucleo
E_nuc = mol.nuclear_repulsion_energy()

Antes de construir la matriz de Fock debemos caluclar las cantidades estáticas del ciclo (no cambian de ciclo a ciclo),
que son el Hamiltoniano de Core, la matriz de Solapamiento y los ERIs. 

Emepcemos por las matrices de 1e-. Primero  hay crear un objeto de función de onda de la molecula que quiero calcular.
Para poder construir el objeto *wfn*, le debo entregar la información de la geometría molecular y la función de base 
atómica:


Esta la  que me calcula las matrices de 1e-. Naturalmente,
la inicio entregandole la información de  la base atómico que voy a utilizar.

Afortunadamente lo podemos hacer utilizando las librerias de  <span style='font-variant: small-caps'> Psi4</span>!



In [25]:
# ==> Ojeto wfn <==
# Construir función de onda
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis')) # La base es la cc-pVTZ

Luego con el objeto *wfn* crear una instancia de MintsHelper que me permite calcular las matrices de integrales (mints).

In [26]:
# Creación de instancia MintsHelper
mints = psi4.core.MintsHelper(wfn.basisset())

# Construir matriz de solapamiento
A partir de mints puedo guardar la matriz de solapamiento **S** en un array de numpy.

In [27]:
# Overlap matrix
S = np.asarray(mints.ao_overlap())

El objeto **S**  creado por *mints*, a su vez tiene atributos. Por ejemplo lo podemos 
imprimir, obtener el tamaño de la matriz ( que es el número de bases atómicas, cuantas son?). El número
de orbitales ocupados, lo podemos obtener del objecto **wfn***

In [28]:
nbf = S.shape
print nbf[0]
ndocc = wfn.nalpha()
print ndocc 

14
2


# Construya el Hamiltoniano de Core

A partir de ao_kinetic y ao_potential, construya el hamiltoniano de core de su molécula

In [29]:
# Build core Hamiltonian
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V

# Tensor ERI

Ahora nos podemos dedicar a construir el tensor ERI. LibMints hace la construcción del tensor extremadamente 
simple. Sin embargo antes de guardar el tensor, revisemos primero el tamaño del tensor para revisar si la memoria que le asignamos a numpy es suficiente. Cada integral de mi tensor es un *double float* que pesa 8 bytes. Por lo tanto el 
tamaño total en MB es:

In [30]:
# Memory check for ERI tensor
I_size = (nbf[0]**4) * 8.e-6
print('\nSize of the ERI tensor will be {:4.2f} MB.'.format(I_size))
memory_footprint = I_size * 1.5

# Build ERI Tensor
I = np.asarray(mints.ao_eri())



Size of the ERI tensor will be 0.31 MB.


Si cambia la base a una base triple zeta, en cuanto va aumentar el requerimiento de memoria de su calculo?


Ahora podemos almacenar el tensor ERI en un array numpy de la misma forma que guardamos la matriz de solapamiento. Revise las dimensiones 

In [31]:
# Tensor con integrales 2e-
I = np.asarray(mints.ao_eri())

# Resulva el problema auto-consitente, y determine la energía electrónica de su molecula. 

In [32]:
# ==> Construct AO orthogonalization matrix A <==
A = mints.ao_overlap()
A.power(-0.5, 1.e-16)
A = np.asarray(A)

# Check orthonormality
S_p = A.dot(S).dot(A)
new_hope = np.allclose(S_p, np.eye(S.shape[0]))

if new_hope:
    print('There is a new hope for diagonalization!')
else:
    print("Whoops...something went wrong. Check that you've correctly built the transformation matrix.")
    
# ==> Compute C & D matrices with CORE guess <==
# Transformed Fock matrix
F_p = A.dot(H).dot(A)

# Diagonalize F_p for eigenvalues & eigenvectors with NumPy
e, C_p = np.linalg.eigh(F_p)

# Transform C_p back into AO basis
C = A.dot(C_p)

# Grab occupied orbitals
C_occ = C[:, :ndocc]

# Build density matrix from occupied orbitals
D = np.einsum('pi,qi->pq', C_occ, C_occ)

# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()

# ==> SCF Iterations <==
# Pre-iteration energy declarations
SCF_E = 0.0
E_old = 0.0

print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrix
    J = np.einsum('pqrs,rs->pq', I, D)
    K = np.einsum('prqs,rs->pq', I, D)
    F = H + 2*J - K
    
    # Compute RHF energy
    SCF_E = np.einsum('pq,pq->', (H + F), D) + E_nuc
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E' % (scf_iter, SCF_E, SCF_E - E_old))
    
    # SCF Converged?
    if (abs(SCF_E - E_old) < E_conv):
        break
    E_old = SCF_E
    
    # Compute new orbital guess
    F_p =  A.dot(F).dot(A)
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :ndocc]
    D = np.einsum('pi,qi->pq', C_occ, C_occ)
    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        psi4.core.clean()
        raise Exception("Maximum number of SCF iterations exceeded.")

# Post iterations
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % (SCF_E))

There is a new hope for diagonalization!
==> Starting SCF Iterations <==

SCF Iteration   1: Energy = -14.4051493525856067 dE = -1.44051E+01
SCF Iteration   2: Energy = -14.5588836684578080 dE = -1.53734E-01
SCF Iteration   3: Energy = -14.5715228221709303 dE = -1.26392E-02
SCF Iteration   4: Energy = -14.5722914423453354 dE = -7.68620E-04
SCF Iteration   5: Energy = -14.5723350524472171 dE = -4.36101E-05
SCF Iteration   6: Energy = -14.5723374875228213 dE = -2.43508E-06
SCF Iteration   7: Energy = -14.5723376229817507 dE = -1.35459E-07

SCF converged.
Final RHF Energy: -14.57233762 [Eh]
